# Air Quality Index (AQI) Data Exploration and Preprocessing
## Dataset: Jakarta Air Quality 2010-2025
### Source: https://www.kaggle.com/datasets/senadu34/air-quality-index-in-jakarta-2010-2021

## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 2. Load Dataset

In [ ]:
# Load the main dataset
df = pd.read_csv('../data/ispu_dki_all.csv')

# Create a backup copy
df_original = df.copy()

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

## 3. Dataset Information

In [ ]:
# Basic information about the dataset
print("=== DATASET OVERVIEW ===")
print(f"Dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\n=== COLUMN INFORMATION ===")
df.info()

print("\n=== FIRST 5 ROWS ===")
display(df.head())

print("\n=== LAST 5 ROWS ===")
display(df.tail())

## 4. Data Wrangling
### 4.1 Gathering Data

In [ ]:
# Data gathering summary
print("=== DATA GATHERING SUMMARY ===")
print(f"Total records: {len(df):,}")
print(f"Date range: {df['tanggal'].min()} to {df['tanggal'].max()}")
print(f"Unique stations: {df['stasiun'].nunique()}")
print(f"Station names: {df['stasiun'].unique()}")

# Check data types
print("\n=== DATA TYPES ===")
print(df.dtypes)

### 4.2 Assessing Data

In [ ]:
# Missing values assessment
print("=== MISSING VALUES ASSESSMENT ===")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
print(missing_df)

# Duplicate assessment
print(f"\n=== DUPLICATE ASSESSMENT ===")
duplicates = df.duplicated().sum()
print(f"Total duplicates: {duplicates}")

# Statistical summary
print("\n=== STATISTICAL SUMMARY ===")
display(df.describe())

### 4.3 Cleaning Data

In [ ]:
# Convert date column to datetime
print("=== DATETIME CONVERSION ===")
df['tanggal'] = pd.to_datetime(df['tanggal'])
print(f"Date column converted to datetime")
print(f"Date range: {df['tanggal'].min()} to {df['tanggal'].max()}")

# Extract date features
df['year'] = df['tanggal'].dt.year
df['month'] = df['tanggal'].dt.month
df['day'] = df['tanggal'].dt.day
df['dayofweek'] = df['tanggal'].dt.dayofweek
df['quarter'] = df['tanggal'].dt.quarter

print("Date features extracted: year, month, day, dayofweek, quarter")

## 5. Missing Values and Preprocessing

In [ ]:
# Handle missing values
print("=== HANDLING MISSING VALUES ===")
numeric_columns = ['pm25', 'pm10', 'so2', 'co', 'o3', 'no2', 'max']

# Fill missing values with median for numeric columns
for col in numeric_columns:
    if col in df.columns:
        missing_before = df[col].isnull().sum()
        df[col] = df[col].fillna(df[col].median())
        print(f"{col}: {missing_before} missing values filled with median")

# Handle categorical missing values
if 'critical' in df.columns:
    df['critical'] = df['critical'].fillna('Unknown')
    print(f"Critical column: missing values filled with 'Unknown'")

if 'category' in df.columns:
    df['category'] = df['category'].fillna('Unknown')
    print(f"Category column: missing values filled with 'Unknown'")

print(f"\nMissing values after cleaning: {df.isnull().sum().sum()}")

## 6. Exploratory Data Analysis (EDA)
### 6.1 Distribution Analysis

In [ ]:
# Distribution of pollutants
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

pollutants = ['pm25', 'pm10', 'so2', 'co', 'o3', 'no2']

for i, pollutant in enumerate(pollutants):
    if pollutant in df.columns:
        axes[i].hist(df[pollutant].dropna(), bins=50, alpha=0.7, edgecolor='black')
        axes[i].set_title(f'Distribution of {pollutant.upper()}')
        axes[i].set_xlabel(pollutant.upper())
        axes[i].set_ylabel('Frequency')
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 6.2 Correlation Analysis

In [ ]:
# Correlation matrix
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Matrix of Air Quality Parameters')
plt.tight_layout()
plt.show()

# Strong correlations
print("=== STRONG CORRELATIONS (>0.5) ===")
strong_corr = correlation_matrix.abs() > 0.5
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if strong_corr.iloc[i, j]:
            print(f"{correlation_matrix.columns[i]} - {correlation_matrix.columns[j]}: {correlation_matrix.iloc[i, j]:.3f}")

### 6.3 Statistical Analysis

In [ ]:
# Statistical analysis
print("=== STATISTICAL ANALYSIS ===")
stats_df = df[pollutants].describe()
display(stats_df)

# Skewness and Kurtosis
print("\n=== SKEWNESS AND KURTOSIS ===")
skew_kurt = pd.DataFrame({
    'Skewness': df[pollutants].skew(),
    'Kurtosis': df[pollutants].kurtosis()
})
display(skew_kurt)

### 6.4 Univariate Analysis

In [ ]:
# Box plots for outlier detection
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, pollutant in enumerate(pollutants):
    if pollutant in df.columns:
        axes[i].boxplot(df[pollutant].dropna())
        axes[i].set_title(f'Box Plot of {pollutant.upper()}')
        axes[i].set_ylabel(pollutant.upper())
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 6.5 Bivariate Analysis

In [ ]:
# Time series analysis
df_monthly = df.groupby(['year', 'month'])[pollutants].mean().reset_index()
df_monthly['date'] = pd.to_datetime(df_monthly[['year', 'month']].assign(day=1))

fig, axes = plt.subplots(3, 2, figsize=(16, 18))
axes = axes.ravel()

for i, pollutant in enumerate(pollutants):
    axes[i].plot(df_monthly['date'], df_monthly[pollutant], marker='o', linewidth=2)
    axes[i].set_title(f'Monthly Average {pollutant.upper()} Over Time')
    axes[i].set_xlabel('Date')
    axes[i].set_ylabel(f'{pollutant.upper()}')
    axes[i].grid(True, alpha=0.3)
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 6.6 Multivariate Analysis

In [ ]:
# Pair plot for multivariate analysis
sample_df = df[pollutants].sample(n=min(1000, len(df)))  # Sample for performance
sns.pairplot(sample_df, diag_kind='hist', plot_kws={'alpha': 0.6})
plt.suptitle('Pairwise Relationships Between Pollutants', y=1.02)
plt.show()

## 7. Advanced Data Preprocessing
### 7.1 Remove Missing Values and Duplicates

In [ ]:
# Remove duplicates
print("=== REMOVING DUPLICATES ===")
before_dup = len(df)
df = df.drop_duplicates()
after_dup = len(df)
print(f"Removed {before_dup - after_dup} duplicate rows")

# Final missing value check
print("\n=== FINAL MISSING VALUE CHECK ===")
print(df.isnull().sum())

### 7.2 Outlier Detection and Handling

In [ ]:
# Outlier detection using IQR method
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

print("=== OUTLIER DETECTION ===")
outlier_summary = {}

for pollutant in pollutants:
    if pollutant in df.columns:
        outliers, lower, upper = detect_outliers_iqr(df, pollutant)
        outlier_summary[pollutant] = {
            'count': len(outliers),
            'percentage': (len(outliers) / len(df)) * 100,
            'lower_bound': lower,
            'upper_bound': upper
        }
        print(f"{pollutant}: {len(outliers)} outliers ({(len(outliers)/len(df)*100):.2f}%)")

# Cap outliers instead of removing them
df_clean = df.copy()
for pollutant in pollutants:
    if pollutant in df_clean.columns:
        lower = outlier_summary[pollutant]['lower_bound']
        upper = outlier_summary[pollutant]['upper_bound']
        df_clean[pollutant] = df_clean[pollutant].clip(lower=lower, upper=upper)

print("\nOutliers capped to IQR bounds")

### 7.3 Feature Scaling and Normalization

In [ ]:
# Feature scaling
scaler = StandardScaler()
scaled_features = pollutants + ['max']
scaled_features = [col for col in scaled_features if col in df_clean.columns]

df_scaled = df_clean.copy()
df_scaled[scaled_features] = scaler.fit_transform(df_clean[scaled_features])

print("=== FEATURE SCALING COMPLETED ===")
print(f"Scaled features: {scaled_features}")
print("\nScaled data statistics:")
display(df_scaled[scaled_features].describe())

### 7.4 Categorical Encoding

In [ ]:
# Encode categorical variables
le_critical = LabelEncoder()
le_category = LabelEncoder()

if 'critical' in df_clean.columns:
    df_clean['critical_encoded'] = le_critical.fit_transform(df_clean['critical'])
    print(f"Critical pollutant encoded: {dict(zip(le_critical.classes_, le_critical.transform(le_critical.classes_)))}")

if 'category' in df_clean.columns:
    df_clean['category_encoded'] = le_category.fit_transform(df_clean['category'])
    print(f"Category encoded: {dict(zip(le_category.classes_, le_category.transform(le_category.classes_)))}")

### 7.5 Feature Binning

In [ ]:
# Create AQI categories based on max values
def categorize_aqi(value):
    if value <= 50:
        return 'Good'
    elif value <= 100:
        return 'Moderate'
    elif value <= 150:
        return 'Unhealthy for Sensitive Groups'
    elif value <= 200:
        return 'Unhealthy'
    elif value <= 300:
        return 'Very Unhealthy'
    else:
        return 'Hazardous'

if 'max' in df_clean.columns:
    df_clean['aqi_category'] = df_clean['max'].apply(categorize_aqi)
    print("=== AQI CATEGORIES CREATED ===")
    print(df_clean['aqi_category'].value_counts())

## 8. Save Cleaned Data

In [ ]:
# Save cleaned datasets
print("=== SAVING CLEANED DATA ===")

# Save original cleaned data
df_clean.to_csv('../data/aqi_cleaned.csv', index=False)
print("Cleaned data saved to: ../data/aqi_cleaned.csv")

# Save scaled data
df_scaled.to_csv('../data/aqi_scaled.csv', index=False)
print("Scaled data saved to: ../data/aqi_scaled.csv")

# Save preprocessing objects
import joblib
joblib.dump(scaler, '../artifacts/scaler.pkl')
if 'critical' in df_clean.columns:
    joblib.dump(le_critical, '../artifacts/label_encoder_critical.pkl')
if 'category' in df_clean.columns:
    joblib.dump(le_category, '../artifacts/label_encoder_category.pkl')

print("Preprocessing objects saved to: ../artifacts/")

# Final summary
print(f"\n=== FINAL DATASET SUMMARY ===")
print(f"Original shape: {df_original.shape}")
print(f"Cleaned shape: {df_clean.shape}")
print(f"Features: {list(df_clean.columns)}")
print(f"Data types: {df_clean.dtypes.value_counts()}")
print("\nData preprocessing completed successfully!")